# 來認識一下 python generator

python 中的 generator 可以方便地讓我們生成 iterator

而 iterator 則可以透過 __next__ 方法來取得 iterator 的下一個值


# Python Generator

此筆記學習 python 生成器的筆記，主要來自 http://lotabout.me/2017/Python-Generator/ 的介紹！

此篇文章介紹的非常棒，所以我決定用我的改寫紀錄一下，方便未來參考回憶！

我們都知道 python 的 generator 非常方便，他實現了 itertor 與 lazy evalution 的功能，

以下介紹其運作原理，話從 iterator 說起！


In [3]:
# iterator
# 假設需要獲取一個有順序的 id 物件
# 我們可以透過這個方法來獲得
import time

def get_id_by_request(i):
    time.sleep(5)
    return i

def get_ids(n):
    id_list = []
    for i in range(n):
        id_list.append(get_id_by_request(i))
    return id_list

images = get_ids(3)


但是 get_id_by_request() 是個非常耗時的工作

希望可以盡量在需要他時在調用它，可以做個小修改

In [7]:
ID = -1

def next_id():
    global ID
    ID += 1
    return get_id_by_request(ID)

id0 = next_id()
id1 = next_id()


這樣一來我們就可以在需要下一個id時在調用這個方法

就可以避免這個調用 get_id_by_request() 很耗時的問題

但是這樣全局變量定義ID後，無法被第二個人使用，於是定義一個 class 來操作它

In [9]:
class GetID:
    def __init__(self):
        self.id = -1
        
    def next_id(self):
        self.id += 1
        return get_id_by_request(self.id)
    
    
id = GetID()
id0 = id.next_id()
id1 = id.next_id()

# 而這就是一個 iterator 的概念
# 我們可以透過實現 __iter__ 方法與 __next__ 來把 GetID 變成一個 python iterator


class GetID:
    def __init__(self):
        self.id = -1
        
    def __iter__(self):
        return self
    
    def __next__(self):
        self.id += 1
        return get_id_by_request(self.id)


for id in GetID():
    print(id)
    if id == 5:
        break


0
1
2
3
4
5


# 從 iterator 到 generator

從上面我們可以知道，我們必須自己定義 iterator 的狀態

寫法上比較不直覺，於是 python 利用了 yeild 這個關鍵字


In [10]:
# generator 改寫

def get_id():
    id = -1
    while True:
        id += 1
        yield get_id_by_request(id)
        
for id in get_id():
    print(id)
    if id == 5:
        break


0
1
2
3
4
5


要寫 generator 其實乍看之下就是把 return 改成 yield

事實上調用 get_id() 這個函數會返回一個 generator object

他實現了 iterator 的方法！

In [11]:
# 比較利用 generator 寫法與字型實現 iterator 方法的 fibonacci 函數：
def fibonacci():
    a, b = (0, 1)
    while True:
        yield a
        a, b = b, a+b

fibos = fibonacci()
next(fibos) #=> 0
next(fibos) #=> 1
next(fibos) #=> 1
next(fibos) #=> 2


class Fibonacci():
    def __init__(self):
        self.a, self.b = (0, 1)
    def __iter__(self):
        return self
    def __next__(self):
        result = self.a
        self.a, self.b = self.b, self.a + self.b
        return result

fibos = Fibonacci()
next(fibos) #=> 0
next(fibos) #=> 1
next(fibos) #=> 1
next(fibos) #=> 2

2

# yield 到底做了什麼？

一個 generator 執行檔 yield 時會保存當前狀態，然後 return

之後會把執行流還給調用的函數，直到下一次在調用這個 generator 時，

會從上次保存狀態的 yield 繼續執行

In [15]:
def generator():
    print('one')
    yield            # break 1
    print('two')
    yield            # break 2
    print('three')

x = generator()

next(x)

next(x)

next(x)

one
two
three


StopIteration: 

我們可以看到第一次調用 next(x) 在 yield 就返回了，

然後繼續執行 next() 方法則是會從 # break1 開始執行到下一個 yield

直到第三次後拋出 StopIteration 的異常。

而這個功能可以用在有順序關係的調用上，讓我們不會因為調用順序錯誤而造成問題！

# yield 能做更多事嗎？

若是我們可以在每一次調用 next 方法時對 yield 傳入參數

在 PEP 342 中加入了相應的支持

In [19]:
# 假設我們想要計算數列上的到每個點以前的平均值
# 我們可以使用 send() 方法來對 yield 傳入參數
def averager():
    sum = 0
    num = 0
    while True:
        sum += (yield sum / num if num > 0 else 0)
        num += 1

x = averager()
x.send(None)  # x 內的 sum=0 num=0 並且停在 yield 處
#=> 0
x.send(1)  # 從 yield 處開始調用，並且傳入 1， 此時 sum=1, num=1 經過迴圈， yield 1，此時狀態 sum=1 num=1
#=> 1.0
x.send(2)  # 從 yield 處開始調用，並且傳入 2， 此時 sum=3, num=2 經過迴圈， yield 1.5，此時狀態 sum=3 num=2
#=> 1.5
x.send(3)
#=> 2.0

2.0

原本的 yield 從語句變成了表達式，所以我們可以把 yield 寫成

x = yield 10, y = 10 + (yield), foo(yield 42)

Python 規定，除非 yield 左邊直接跟著等號，否則必須用括號括起來！

當 Python 執行到 yield 表達式時，它首先計算 yield 右邊的表達式，
上例中即為 sum / num if num > 0 else 0 的值，暫停當前的控制流，並返回。

之後，除了可以用 next(generator) 的方式（即 iterator 的方式）來恢復控制流之外，
還可以通過 generator.send(some_value) 來傳遞一些值。

例如上例中，如果我們調用 x.send(3) 則 Python 恢復控制流， 
(yield sum/sum ...) 的值則為我們賦予的 3，並接著執行 sum += 3 以及之後的語句。
注意的是，如果這時我們用的是 next(generator) 則它等價為 generator.send(None)。

最後要注意的是，剛調用generator 生成 generator object 時，函數並沒有真正運行，
也就是說這時控制流並不在yield 表達式上等待用戶傳遞值，
因此我們需要先調用generate.send(None) 或next(generator) 來觸發最開始的執行。

In [20]:
class Averager:
    def __init__(self):
        self.sum = 0
        self.num = 0
    def avg_num(self, n):
        self.sum += n
        self.num += 1
        return self.sum / self.num
averager = Averager()
averager.avg_num(1)
#=> 1.0
averager.avg_num(2)
#=> 1.5
averager.avg_num(3)
#=> 2.0

2.0

我們在看到上面使用的方法，明顯比 generator 直覺許多，

所以用 send() 方法 對 yield 傳入值，主要並不是要用在這裡的。

他真正的需求是 "coroutine(協程)" 用來實現 python 的異步編程！ 

# yield from

考慮我們有多個 generator 並想把 generator 組合起來，如：

In [2]:
def odds(n):
    for i in range(n):
        if i % 2 == 1:
            yield i

def evens(n):
    for i in range(n):
        if i % 2 == 0:
            yield i

def odd_even(n):
    for x in odds(n):
        yield x
    for x in evens(n):
        yield x

for x in odd_even(6):
    print(x)
#=> 1, 3, 5, 0, 2, 4

1
3
5
0
2
4


for x in generator(): yield x 這種寫法不太方便，

因此 PEP 380 引入了 yield from 語法，

來替代我們前面說的這種語法，因此上面的例子可以改成：

In [3]:
def odd_even(n):
    yield from odds(n)
    yield from evens(n)
    
for x in odd_even(6):
    print(x)

1
3
5
0
2
4


In [7]:
# 練習1

def func(n):
    return list(range(n))

def gene(n):
    for i in range(n):
        yield i

print(func, gene)
print(func(100), gene(100))
print(list(gene(100)))

<function func at 0x109b0cb70> <function gene at 0x109b0c048>
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99] <generator object gene at 0x109ae4eb8>
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [34]:
# 練習2 

NoneType = type(None)
def guess_string():
    s = 'a'
    yield
    s += 'b'
    yield s
    while True:
        s = s[::-1]
        s = yield s + 'c'
        if isinstance(s, NoneType):
            s = 'n'
        s += 'd'
    
gs = guess_string()
print(gs.__next__())  # None
print(gs.__next__())  # ab
print(gs.__next__())  # bac
print(gs.__next__())  # dnc
print(gs.__next__())  # dnc
print(gs.send(None))  # dnc
print(gs.send('p'))   # dpc
print(gs.send('gg'))  # dggc

None
ab
bac
dnc
dnc
dnc
dpc
dggc


In [52]:
def odds(n):
    for i in range(n):
        print(i, 'in_odds')
        if i % 2 == 1:
            yield i

def evens(n):
    for i in range(n):
        print(i, 'in_evens')
        if i % 2 == 0:
            yield i

def odd_even(n):
    print(n, 'in_odd_even')
    for x in odds(n):
        print(x, 'in_odd_even_odds')
        yield x
    for x in evens(n):
        print(x, 'in_odd_even_evens')
        yield x
    

for x in odd_even(6):
    print(x)
#=> 1, 3, 5, 0, 2, 4

6 in_odd_even
0 in_odds
1 in_odds
1 in_odd_even_odds
1
2 in_odds
3 in_odds
3 in_odd_even_odds
3
4 in_odds
5 in_odds
5 in_odd_even_odds
5
0 in_evens
0 in_odd_even_evens
0
1 in_evens
2 in_evens
2 in_odd_even_evens
2
3 in_evens
4 in_evens
4 in_odd_even_evens
4
5 in_evens


In [54]:
def odds(n):
    for i in range(n):
        print(i, 'in_odds')
        if i % 2 == 1:
            yield i

def evens(n):
    for i in range(n):
        print(i, 'in_evens')
        if i % 2 == 0:
            yield i
        
def odd_even2(n):
    print(n, 'in_odd_even2')
    yield from odds(n)
    print('pass1')
    yield from evens(n)
    print('pass2')

for x in odd_even2(6):
    print(x)
#=> 1, 3, 5, 0, 2, 4

6 in_odd_even2
0 in_odds
1 in_odds
1
2 in_odds
3 in_odds
3
4 in_odds
5 in_odds
5
pass1
0 in_evens
0
1 in_evens
2 in_evens
2
3 in_evens
4 in_evens
4
5 in_evens
pass2
